In [25]:
import os
import time
from selenium.webdriver import Firefox, FirefoxOptions, FirefoxService
import pandas as pd

In [26]:
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

In [27]:
options = FirefoxOptions()
options.profile = os.environ['FIREFOX_PROFILE_PATH']
service = FirefoxService(executable_path='C:\\Users\\leowy\\anaconda3\\envs\\market-returns\\Scripts\\geckodriver.exe')

In [28]:
driver = Firefox(options=options, service=service)
driver.implicitly_wait(0.5)

In [29]:
driver.get('https://markets.ft.com/data/indices/tearsheet/historical?s=TFTSTIU:FSI')

In [30]:
driver.find_element(by='class name', value='mod-ui-filter-overlay__filter-toggle').click()

In [31]:
def select_start_date(driver: Firefox, year: int):
    driver.find_elements(by='class name', value='picker__input')[0].click()
    
    start_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/select')
    start_year_select = Select(start_year)
    if int(start_year_select.options[0].text) > year:
        start_year.click()
        start_year_select.select_by_index(0)
        
    start_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/select')
    start_year_select = Select(start_year)
    start_year.click()
    start_year_select.select_by_value(f'{year}')
    
    for i in range(15):
        start_month = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/div[1]')
        if start_month.text == 'January':
            break
        start_prev = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/div[2]')
        start_prev.click()
        
    start_date_cell = driver.find_element(by='css selector', value=f'div[aria-label="1 Jan, {year}"]')
    start_date_cell.click()

In [32]:
def select_end_date(driver: Firefox, year: int):
    end_date_field = driver.find_elements(by='class name', value='picker__input')[1]
    end_date_field.click()
    
    end_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/select')
    end_year_select = Select(end_year)
    if int(end_year_select.options[0].text) > year:
        end_year.click()
        end_year_select.select_by_index(0)
    
    end_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/select')
    end_year_select = Select(end_year)
    end_year.click()
    end_year_select.select_by_value(f'{year}')
    
    if year == pd.to_datetime('today').year:
        end_date = pd.to_datetime('today').strftime('%#d %b, %Y')
        end_month_str = pd.to_datetime('today').strftime('%B')
    else:
        end_date = f'31 Dec, {year}'
        end_month_str = 'December'
    
    for i in range(15):
        end_month = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/div[1]')
        if end_month.text == end_month_str:
            break
        end_next = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/div[3]')
        end_next.click()
        
    end_date_cell = driver.find_element(by='css selector', value=f'div[aria-label="{end_date}"]')
    end_date_cell.click()

In [33]:
rows = []

for year in range(2008, pd.to_datetime('today').year + 1):
    
    if year == 2008:
        select_start_date(driver, year)
        WebDriverWait(driver, 10).until(EC.invisibility_of_element(('class name', 'mod-ui-loading__overlay')))
        select_end_date(driver, year)
    
    else:
        select_end_date(driver, year)
        WebDriverWait(driver, 10).until(EC.invisibility_of_element(('class name', 'mod-ui-loading__overlay')))
        select_start_date(driver, year)

    WebDriverWait(driver, 10).until(EC.invisibility_of_element(('class name', 'mod-ui-loading__overlay')))
    
    rows.extend(driver.find_element(by='class name', value='mod-tearsheet-historical-prices__results').text.split('\n')[:0:-1])

In [34]:
sti = pd.DataFrame([row.rsplit(' ', maxsplit=5) for row in rows[1:]], columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [35]:
sti['Date'] = sti['Date'].apply(pd.to_datetime)

In [36]:
sti = sti.replace(',','', regex=True)

In [37]:
sti[['Open', 'High', 'Low', 'Close', 'Volume']] = sti[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

In [38]:
sti.set_index('Date', drop=True).to_csv('data/Straits Times Index USD Gross.csv')

In [39]:
driver.quit()